In [1]:
#!pip install simple_salesforce
#!pip install pymysql
#!pip install sqlalchemy

In [2]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [3]:
# Make sure to use your own `config.py` file. Consider ensuring that these variable names are in sync
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [4]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [5]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Prepare ETL for the Course data

In [6]:
course_data_df = pd.read_sql("SELECT * FROM course", conn)
course_data_df.head(5)

,ID_Course,CourseCode,CourseName,CreditHours,BootCampCourse
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [7]:
course_data_df.rename(columns={
    'CourseCode':'Course_Code__c',
    'CourseName':'Course_Name__c',
    'CreditHours':'Credit_Hours__c',
    'BootCampCourse':'Boot_Camp_Course__c',    
}, inplace=True)
course_data_df.head()

,ID_Course,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [8]:
course_data_df = course_data_df[['Course_Code__c', 'Course_Name__c', 'Credit_Hours__c', 'Boot_Camp_Course__c']]
course_data_df.head()

,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,BC-WEBDEV,Full Stack Web Development,12,1
2,BC-UIUX,User Interface/User Experience,12,1
3,CIS-349,Introduction to Databases,5,0
4,CIS-405,Database Programming,5,0


In [9]:
course_data_records = course_data_df.to_dict('records')
course_data_records

[{'Course_Code__c': 'BC-DATAVIZ',
  'Course_Name__c': 'Data Visualization and Analytics',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'BC-WEBDEV',
  'Course_Name__c': 'Full Stack Web Development',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'BC-UIUX',
  'Course_Name__c': 'User Interface/User Experience',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'CIS-349',
  'Course_Name__c': 'Introduction to Databases',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-405',
  'Course_Name__c': 'Database Programming',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-438',
  'Course_Name__c': 'Database Administration',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-430',
  'Course_Name__c': 'Business Systems Programming I',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-435',
  'Course_Name__c':

In [10]:
#Deleting courses records
course_records = sf.query("SELECT Name FROM Course__c")
recs_to_delete = [{'Name': r['Name']} for r in course_records['records']]
recs_to_delete

[{'Name': 'a004x000003VtNK'},
 {'Name': 'a004x000003VtNt'},
 {'Name': 'a004x000003VtNe'},
 {'Name': 'a004x000003VtNZ'},
 {'Name': 'a004x000003VtNP'},
 {'Name': 'a004x000003VtNU'},
 {'Name': 'a004x000003VtNj'},
 {'Name': 'a004x000003VtNo'}]

### Deleting from `Course` Record

In [11]:
for rec in recs_to_delete:
    try:
        sf.Course__c.delete(rec['Name'])
    except Exception as e:
        print(e)

Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Course__c/a004x000003VtNK. Response content: [{'message': 'Your attempt to delete a004x000003VtNK could not be completed because it is associated with the following class participants.: a094x000000ancY, a094x000000ancd, a094x000000anci, a094x000000ancn, ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ...\n', 'errorCode': 'DELETE_FAILED', 'fields': []}]


In [12]:
# Example of insurting in Bulk 
# sf.bulk.Course__c.insert(course_data_records)

### Inserting Data into `course` Record in SF

In [13]:

for rec in course_data_records:

    record = {
        'Course_Code__c': rec['Course_Code__c'],
        'Course_Name__c': rec['Course_Name__c'],
        'Credit_Hours__c': rec['Credit_Hours__c'],
        'Boot_Camp_Course__c': rec['Boot_Camp_Course__c'],
    }
    
    try:
        sf.Course__c.create(record)
    except Exception as e:
        print(e)

Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Course__c/. Response content: [{'message': 'duplicate value found: Course_Code__c duplicates value on record with id: a004x000003VtNK', 'errorCode': 'DUPLICATE_VALUE', 'fields': []}]


### Create `Course` Lookup Table
You will use this later to crosswalk the course code with the primary key from the `Course` table

It is important to note that we will be querying **Salesforce** to retrieve the record IDs 

In [14]:
course_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT Name, Course_Code__c FROM Course__c")
for row in data:
    rec = {
        'ID_Course__c': row['Name'], # this is a critical line of code
        'Course_Code__c': row['Course_Code__c']
    }
    course_lookup_list.append(rec)
    
course_lookup_list

[{'ID_Course__c': 'a004x000003Vusa', 'Course_Code__c': 'CIS-349'},
 {'ID_Course__c': 'a004x000003VusV', 'Course_Code__c': 'BC-UIUX'},
 {'ID_Course__c': 'a004x000003Vusf', 'Course_Code__c': 'CIS-405'},
 {'ID_Course__c': 'a004x000003Vusp', 'Course_Code__c': 'CIS-430'},
 {'ID_Course__c': 'a004x000003Vusu', 'Course_Code__c': 'CIS-435'},
 {'ID_Course__c': 'a004x000003VusQ', 'Course_Code__c': 'BC-WEBDEV'},
 {'ID_Course__c': 'a004x000003Vusk', 'Course_Code__c': 'CIS-438'},
 {'ID_Course__c': 'a004x000003VtNK', 'Course_Code__c': 'BC-DATAVIZ'}]

In [15]:
#Convert into DataFrame 
course_lookup_df = pd.DataFrame(course_lookup_list)
course_lookup_df

,ID_Course__c,Course_Code__c
0,a004x000003Vusa,CIS-349
1,a004x000003VusV,BC-UIUX
2,a004x000003Vusf,CIS-405
3,a004x000003Vusp,CIS-430
4,a004x000003Vusu,CIS-435
5,a004x000003VusQ,BC-WEBDEV
6,a004x000003Vusk,CIS-438
7,a004x000003VtNK,BC-DATAVIZ


### Query the `Class` table from MySQL


In [16]:
# Query the `Class` table from MySQL
query = '''
   SELECT 
        co.CourseCode AS Course_Code__c,
        cl.Section AS Section__c,
        cl.StartDate AS Start_Date__c,
        cl.EndDate AS End_Date__c       
    FROM 
        class cl
        INNER JOIN course co
        ON cl.ID_Course = co.ID_Course

'''

class_data_df = pd.read_sql(query, conn)
class_data_df.head()

,Course_Code__c,Section__c,Start_Date__c,End_Date__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08
2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07
3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07


In [17]:

class_records = class_data_df.to_dict(orient='records')
class_records

[{'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWU-ARL-DATA-PT-09-0',
  'Start_Date__c': datetime.date(2020, 9, 16),
  'End_Date__c': datetime.date(2020, 3, 14)},
 {'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWDC201805DATA3',
  'Start_Date__c': datetime.date(2018, 5, 15),
  'End_Date__c': datetime.date(2018, 11, 8)},
 {'Course_Code__c': 'BC-WEBDEV',
  'Section__c': 'GWARL201905WEB3',
  'Start_Date__c': datetime.date(2019, 5, 14),
  'End_Date__c': datetime.date(2019, 11, 7)},
 {'Course_Code__c': 'BC-UIUX',
  'Section__c': 'GWARL201905UIUX3',
  'Start_Date__c': datetime.date(2019, 5, 14),
  'End_Date__c': datetime.date(2019, 11, 7)}]

In [18]:
#rename columns from class table
class_data_df.rename(columns={
    'CourseCode':'Course_Code__c',
    'Section':'Section__c',
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c',
    'CourseCode':'Course_Code__c'
}, inplace=True)


class_data_df = class_data_df[['Course_Code__c', 'Section__c', 'Start_Date__c', 'End_Date__c']]
class_data_df

,Course_Code__c,Section__c,Start_Date__c,End_Date__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08
2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07
3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07


### Join the `Class` DataFrame with the `Course` lookup table
This join is necessary to successfully lookup the foreign key for the Course table 

In [122]:
class_xwalk_df = pd.merge(class_data_df, course_lookup_df, how='left')
#class_data_df.drop(columns = ['ID_Class','ID_Course','CourseName','CreditHours','BootCampCourse','Course_Code__c'], inplace=True)

class_xwalk_df.head()


#class_data_df
class_xwalk_df['Start_Date__c'] = class_xwalk_df['Start_Date__c'].astype(str)
class_xwalk_df['End_Date__c'] = class_xwalk_df['End_Date__c'].astype(str)
class_xwalk_df

,Course_Code__c,Section__c,Start_Date__c,End_Date__c,ID_Course__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14,a004x000003VtNK
1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08,a004x000003VtNK
2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07,a004x000003VusQ
3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07,a004x000003VusV


In [123]:
class_xwalk_recs = class_xwalk_df.to_dict(orient='records')
class_xwalk_recs

[{'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWU-ARL-DATA-PT-09-0',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': '2020-03-14',
  'ID_Course__c': 'a004x000003VtNK'},
 {'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWDC201805DATA3',
  'Start_Date__c': '2018-05-15',
  'End_Date__c': '2018-11-08',
  'ID_Course__c': 'a004x000003VtNK'},
 {'Course_Code__c': 'BC-WEBDEV',
  'Section__c': 'GWARL201905WEB3',
  'Start_Date__c': '2019-05-14',
  'End_Date__c': '2019-11-07',
  'ID_Course__c': 'a004x000003VusQ'},
 {'Course_Code__c': 'BC-UIUX',
  'Section__c': 'GWARL201905UIUX3',
  'Start_Date__c': '2019-05-14',
  'End_Date__c': '2019-11-07',
  'ID_Course__c': 'a004x000003VusV'}]

### Deleting from `Class` Record

In [124]:
class_records = sf.query("SELECT Name FROM Class__c")
recs_to_delete = [{'Name': r['Name']} for r in class_records['records']]
recs_to_delete

[{'Name': 'a014x00000A2cwz'},
 {'Name': 'a014x00000A2cwu'},
 {'Name': 'a014x00000A2cwp'},
 {'Name': 'a014x00000A2cx4'},
 {'Name': 'a014x00000A2WQx'}]

In [125]:
for rec in recs_to_delete:
    try:
        sf.class__c.delete(rec['Name'])
    except Exception as e:
        print(e)

Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/class__c/a014x00000A2cwp. Response content: [{'message': 'Your attempt to delete a014x00000A2cwp could not be completed because it is associated with the following class participants.: a094x000000anmj, a094x000000anmt, a094x000000ann3, a094x000000annD, ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ...\n', 'errorCode': 'DELETE_FAILED', 'fields': []}]
Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/class__c/a014x00000A2WQx. Response content: [{'message': 'Your attempt to delete a014x00000A2WQx could not be completed because it is associated with the following class participants.: a094x000000anmo, a094x000000anmy, a094x000000ann8, a094x000000annI, ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ..., ...\n', 'errorCod

## Insert `Class` Records into Salesforce

In [24]:
for rec in class_xwalk_recs:
 
    record = {
        'ID_Course__c': rec['ID_Course__c'],
        'Section__c': rec['Section__c'],
        'Start_Date__c': rec['Start_Date__c'],
        'End_Date__c': rec['End_Date__c'],
    }
    
    try:
        sf.Class__C.create(record)
    except Exception as e:
        print(e)

## Prepare `Student` Data

In [25]:
student_data_df = pd.read_sql("SELECT * FROM student", conn)
student_data_df.head()

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Dartling,Heather,Alice,None,F
1,34,25003514,Dartling,Lana,Cecille,None,F
2,35,25005833,Dartling,Jessica,Dorothy,None,F
3,36,25002589,Dartling,Kimberly,Genevieve,None,F
4,37,25007185,Dartling,Katherine,Lynnette,None,F


In [26]:
student_data_df.rename(columns={
    'StudentID':'Student_ID__c',
    'LastName':'Last_Name__c',
    'FirstName':'First_Name__c',
    'MiddleName':'Middle_Name__c',
    'BirthDate':'Birth_Date__c',
    'Gender':'Gender__c',
}, inplace=True)
student_data_df.head()

,ID_Student,Student_ID__c,Last_Name__c,First_Name__c,Middle_Name__c,Birth_Date__c,Gender__c
0,33,25004961,Dartling,Heather,Alice,None,F
1,34,25003514,Dartling,Lana,Cecille,None,F
2,35,25005833,Dartling,Jessica,Dorothy,None,F
3,36,25002589,Dartling,Kimberly,Genevieve,None,F
4,37,25007185,Dartling,Katherine,Lynnette,None,F


In [29]:
student_data_df = student_data_df[['Student_ID__c', 'Last_Name__c', 'First_Name__c', 'Middle_Name__c', 'Birth_Date__c', 'Gender__c']]
student_data_df.head()

,Student_ID__c,Last_Name__c,First_Name__c,Middle_Name__c,Birth_Date__c,Gender__c
0,25004961,Dartling,Heather,Alice,None,F
1,25003514,Dartling,Lana,Cecille,None,F
2,25005833,Dartling,Jessica,Dorothy,None,F
3,25002589,Dartling,Kimberly,Genevieve,None,F
4,25007185,Dartling,Katherine,Lynnette,None,F


In [30]:
student_data_records = student_data_df.to_dict('records')
student_data_records

[{'Student_ID__c': '25004961',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Heather',
  'Middle_Name__c': 'Alice',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25003514',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Lana',
  'Middle_Name__c': 'Cecille',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25005833',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Jessica',
  'Middle_Name__c': 'Dorothy',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25002589',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Kimberly',
  'Middle_Name__c': 'Genevieve',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25007185',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Katherine',
  'Middle_Name__c': 'Lynnette',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25006014',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Precious',
  'Middle_Name__c': 'Mariette',
  'Birth_Date__c': None,
  'Gender_

### Deleting from `student` Record

In [31]:
student_records = sf.query("SELECT Name FROM Student__c")
recs_to_delete = [{'Name': r['Name']} for r in student_records['records']]
recs_to_delete

[{'Name': 'a064x000004SnCY'},
 {'Name': 'a064x000004SnBV'},
 {'Name': 'a064x000004SnB1'},
 {'Name': 'a064x000004SnB6'},
 {'Name': 'a064x000004SnBG'},
 {'Name': 'a064x000004SnBL'},
 {'Name': 'a064x000004SnBQ'},
 {'Name': 'a064x000004SnBa'},
 {'Name': 'a064x000004SnBk'},
 {'Name': 'a064x000004SnBu'},
 {'Name': 'a064x000004SnBz'},
 {'Name': 'a064x000004SnC4'},
 {'Name': 'a064x000004SnCE'},
 {'Name': 'a064x000004SnCO'},
 {'Name': 'a064x000004SnCT'},
 {'Name': 'a064x000004SnCd'},
 {'Name': 'a064x000004SnCi'},
 {'Name': 'a064x000004SnCs'},
 {'Name': 'a064x000004SnCx'},
 {'Name': 'a064x000004SnD2'},
 {'Name': 'a064x000004SnDC'},
 {'Name': 'a064x000004SnBp'},
 {'Name': 'a064x000004SnC9'},
 {'Name': 'a064x000004SnDH'},
 {'Name': 'a064x000004SnBB'},
 {'Name': 'a064x000004SnCn'},
 {'Name': 'a064x000004SnBf'},
 {'Name': 'a064x000004SnD7'},
 {'Name': 'a064x000004SnDM'},
 {'Name': 'a064x000004SnCJ'}]

In [32]:
for rec in recs_to_delete:
    try:
        sf.Student__c.delete(rec['Name'])
    except Exception as e:
        print(e)

Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/a064x000004SnBV. Response content: [{'message': 'Your attempt to delete a064x000004SnBV could not be completed because it is associated with the following class participants.: a094x000000and7, a094x000000anfI\n', 'errorCode': 'DELETE_FAILED', 'fields': []}]
Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/a064x000004SnB1. Response content: [{'message': 'Your attempt to delete a064x000004SnB1 could not be completed because it is associated with the following class participants.: a094x000000ancY, a094x000000aneo\n', 'errorCode': 'DELETE_FAILED', 'fields': []}]
Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/a064x000004SnB6. Response content: [{'message': 'Your attempt to delete a064x000004SnB6 could not be completed because it is associated with the following class

Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/a064x000004SnBf. Response content: [{'message': 'Your attempt to delete a064x000004SnBf could not be completed because it is associated with the following class participants.: a094x000000andH, a094x000000anfS\n', 'errorCode': 'DELETE_FAILED', 'fields': []}]
Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/a064x000004SnD7. Response content: [{'message': 'Your attempt to delete a064x000004SnD7 could not be completed because it is associated with the following class participants.: a094x000000aneU, a094x000000angf\n', 'errorCode': 'DELETE_FAILED', 'fields': []}]
Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/a064x000004SnDM. Response content: [{'message': 'Your attempt to delete a064x000004SnDM could not be completed because it is associated with the following class

### Insert `student` Records into Salesforce

In [35]:
for rec in student_data_records:

    record = {
        'Student_ID__c': rec['Student_ID__c'],
        'Last_Name__c': rec['Last_Name__c'],
        'First_Name__c': rec['First_Name__c'],
        'Middle_Name__c': rec['Middle_Name__c'],
        'Birth_Date__c': rec['Birth_Date__c'],
        'Gender__c': rec['Gender__c']
    }
    
    try:
        sf.Student__c.create(record)
    except Exception as e:
        print(e)

Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/. Response content: [{'message': 'duplicate value found: Student_ID__c duplicates value on record with id: a064x000004SnB1', 'errorCode': 'DUPLICATE_VALUE', 'fields': []}]
Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/. Response content: [{'message': 'duplicate value found: Student_ID__c duplicates value on record with id: a064x000004SnB6', 'errorCode': 'DUPLICATE_VALUE', 'fields': []}]
Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/. Response content: [{'message': 'duplicate value found: Student_ID__c duplicates value on record with id: a064x000004SnBB', 'errorCode': 'DUPLICATE_VALUE', 'fields': []}]
Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/. Response content: [{'message': 'duplicate value fo

### Prepare The Staff Data

In [164]:
staff_data_df = pd.read_sql("SELECT * FROM staff", conn)
staff_data_df.head(30)

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Luongo,Darick,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Rodney,Heather,Nicole,1994-06-07


In [165]:
#Rename Staff Data
staff_data_df.rename(columns={
    'EmployeeID':'Employee_ID__c',
    'LastName':'Last_Name__c',
    'FirstName':'First_Name__c',
    'MiddleName':'Middle_Name__c', 
    'BirthDate':'Birth_Date__c',
}, inplace=True)

staff_data_df = staff_data_df[['Employee_ID__c', 'Last_Name__c', 'First_Name__c', 'Middle_Name__c', 'Birth_Date__c']]
staff_data_df.head()

,Employee_ID__c,Last_Name__c,First_Name__c,Middle_Name__c,Birth_Date__c
0,000184220,Luongo,Darick,Nico,1995-07-05
1,000130109,Sanford,Gemini,Blair,1992-04-22
2,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,000159108,Rodney,Heather,Nicole,1994-06-07


In [166]:
#pull staff data records
staff_data_records = staff_data_df.to_dict('records')
staff_data_records


[{'Employee_ID__c': '000184220',
  'Last_Name__c': 'Luongo',
  'First_Name__c': 'Darick',
  'Middle_Name__c': 'Nico',
  'Birth_Date__c': datetime.date(1995, 7, 5)},
 {'Employee_ID__c': '000130109',
  'Last_Name__c': 'Sanford',
  'First_Name__c': 'Gemini',
  'Middle_Name__c': 'Blair',
  'Birth_Date__c': datetime.date(1992, 4, 22)},
 {'Employee_ID__c': '000160655',
  'Last_Name__c': 'Williams',
  'First_Name__c': 'Dartanion',
  'Middle_Name__c': 'De Angelo',
  'Birth_Date__c': datetime.date(1993, 5, 21)},
 {'Employee_ID__c': '000159108',
  'Last_Name__c': 'Rodney',
  'First_Name__c': 'Heather',
  'Middle_Name__c': 'Nicole',
  'Birth_Date__c': datetime.date(1994, 6, 7)}]

### Deleting `Staff` record

In [167]:
staff_records = sf.query("SELECT Name FROM staff__c")
recs_to_delete = [{'Name': r['Name']} for r in staff_records['records']]
recs_to_delete

[{'Name': 'a054x0000010hgY'},
 {'Name': 'a054x0000010hgJ'},
 {'Name': 'a054x0000010hgO'},
 {'Name': 'a054x0000010hgT'}]

In [168]:
for rec in recs_to_delete:
    try:
        sf.Staff__c.delete(rec['Name'])
    except Exception as e:
        print(e)

### Instering Data Insto `Staff` Table in SF

In [169]:
for rec in staff_data_records:

    record = {
        'Employee_ID__c': rec['Employee_ID__c'],
        'Last_Name__c': rec['Last_Name__c'],
        'First_Name__c': rec['First_Name__c'],
        'Middle_Name__c': rec['Middle_Name__c'],
        'Birth_Date__c': str(rec['Birth_Date__c']),
        
    }
    
    try:
        sf.staff__c.create(record)
    except Exception as e:
        print(e)


## Query `Student` Records from MySQL

In [170]:
query = '''
    SELECT 
        StudentID AS StudentID__c,
        LastName AS LastName__c,
        FirstName AS FirstName__c,
        MiddleName AS MiddleName__c,
        BirthDate AS BirthDate__c,
        Gender AS Gender__c
    FROM
        student
'''

student_df = pd.read_sql(query, conn)


student_dict = student_df.to_dict(orient='records')

student_dict

[{'StudentID__c': '25004961',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Heather',
  'MiddleName__c': 'Alice',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25003514',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Lana',
  'MiddleName__c': 'Cecille',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25005833',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Jessica',
  'MiddleName__c': 'Dorothy',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25002589',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Kimberly',
  'MiddleName__c': 'Genevieve',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25007185',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Katherine',
  'MiddleName__c': 'Lynnette',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25006014',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Precious',
  'MiddleName__c': 'Mariette',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '

### Deleting `classparticipant` recorda

In [171]:
class_participant_records_del = sf.query("SELECT Name FROM Class_Participant__c")
recs_to_delete = [{'Name': r['Name']} for r in class_participant_records_del['records']]
recs_to_delete

[]

In [172]:
for rec in recs_to_delete:
    try:
        sf.Class_Participant__c.delete(rec['Name'])
    except Exception as e:
        print(e)

### Create `Class` Lookup and Load `Classparticipants` Data into SF

In [173]:
sf_course_records = []

sf_course_data = sf.query('SELECT Name, Course_Code__c FROM Course__c')

for row in sf_course_data['records']:
    
    rec = {
        'ID_Course__c': row['Name'],
        'Course_Code__c': row['Course_Code__c']
    }
    
    sf_course_records.append(rec)
    
sf_course_df = pd.DataFrame(sf_course_records)
sf_course_df.head()

,ID_Course__c,Course_Code__c
0,a004x000003Vusa,CIS-349
1,a004x000003VusV,BC-UIUX
2,a004x000003Vusf,CIS-405
3,a004x000003Vusp,CIS-430
4,a004x000003Vusu,CIS-435


In [174]:
sf_class_records = []

sf_class_data = sf.query('SELECT Name, ID_Course__c, Section__c FROM Class__c')

for row in sf_class_data['records']:
    
    rec = {
        'ID_Class__c': row['Name'],
        'ID_Course__c': row['ID_Course__c'],
        'Section__c': row['Section__c']
    }
    
    sf_class_records.append(rec)
 
sf_class_records



[{'ID_Class__c': 'a014x00000A2cwp',
  'ID_Course__c': 'a004x000003VtNKAA0',
  'Section__c': 'GWU-ARL-DATA-PT-09-0'},
 {'ID_Class__c': 'a014x00000A2WQx',
  'ID_Course__c': 'a004x000003VtNKAA0',
  'Section__c': 'GWU-ARL-DATA-PT-09-0'}]

In [159]:
sf_class_df = pd.DataFrame(sf_class_records)

#sf_class_df['ID_Course__c'] = sf_class_df['ID_Course__c'].apply(lambda id_course: id_course[:-3])

sf_class_df.head()


,ID_Class__c,ID_Course__c,Section__c
0,a014x00000A2cwp,a004x000003VtNK,GWU-ARL-DATA-PT-09-0
1,a014x00000A2WQx,a004x000003VtNK,GWU-ARL-DATA-PT-09-0


In [160]:
sf_class_xwalk = pd.merge(sf_class_df, sf_course_df)
sf_class_xwalk

,ID_Class__c,ID_Course__c,Section__c,Course_Code__c
0,a014x00000A2cwp,a004x000003VtNK,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
1,a014x00000A2WQx,a004x000003VtNK,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ


In [161]:
sf_student_records = []

sf_student_data = sf.query('SELECT Name, Student_ID__c FROM Student__c')

for row in sf_student_data['records']:
    
    rec = {
        'ID_Student__c': row['Name'],
        'Student_ID__c': row['Student_ID__c']
    }
    
    sf_student_records.append(rec)
    
sf_student_xwalk = pd.DataFrame(sf_student_records)
sf_student_xwalk.head()

,ID_Student__c,Student_ID__c
0,a064x000004SnbW,25002555
1,a064x000004Snax,25002311
2,a064x000004SnBV,25007528
3,a064x000004SnB1,25004961
4,a064x000004SnB6,25003514


In [162]:
query = '''

    SELECT
        s.StudentID AS Student_ID__c,
        co.CourseCode AS Course_Code__c,
        cl.Section AS Section__c,
        cp.StartDate AS Start_Date__c,
        cp.EndDate AS End_Date__c
    FROM 
        classparticipant cp
        INNER JOIN class cl
        ON cp.ID_Class = cl.ID_Class
        INNER JOIN course co
        ON cl.ID_Course = co.ID_Course
        INNER JOIN student s
        ON cp.ID_Student = s.ID_Student

'''

class_particiant_df = pd.read_sql(query, conn)
class_particiant_df.head()

,Student_ID__c,Course_Code__c,Section__c,Start_Date__c,End_Date__c
0,25004961,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
1,25003514,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
2,25005833,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
3,25007334,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
4,25002589,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None


In [163]:
class_participant_df= pd.merge(class_particiant_df, sf_class_xwalk, on=['Course_Code__c', 'Section__c'])
class_participant_df.head()


,Student_ID__c,Course_Code__c,Section__c,Start_Date__c,End_Date__c,ID_Class__c,ID_Course__c
0,25004961,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a014x00000A2cwp,a004x000003VtNK
1,25004961,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a014x00000A2WQx,a004x000003VtNK
2,25003514,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a014x00000A2cwp,a004x000003VtNK
3,25003514,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a014x00000A2WQx,a004x000003VtNK
4,25005833,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a014x00000A2cwp,a004x000003VtNK


In [62]:
class_participant_load = pd.merge(class_participant_df, sf_student_xwalk, how='left', on='Student_ID__c')
class_participant_load = class_participant_load[['ID_Student__c', 'ID_Class__c', 'Start_Date__c', 'End_Date__c' ]]
class_participant_load.head()


,ID_Student__c,ID_Class__c,Start_Date__c,End_Date__c
0,a064x000004SnB1,a014x00000A2cwp,2020-09-16,None
1,a064x000004SnB1,a014x00000A2WQx,2020-09-16,None
2,a064x000004SnB6,a014x00000A2cwp,2020-09-16,None
3,a064x000004SnB6,a014x00000A2WQx,2020-09-16,None
4,a064x000004SnBB,a014x00000A2cwp,2020-09-16,None


In [63]:
class_participant_load['Start_Date__c'] = class_participant_load['Start_Date__c'].astype(str)
class_participant_load['End_Date__c'] = class_participant_load['End_Date__c']

In [64]:
class_participant_records = class_participant_load.to_dict(orient='records')
class_participant_records

[{'ID_Student__c': 'a064x000004SnB1',
  'ID_Class__c': 'a014x00000A2cwp',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a064x000004SnB1',
  'ID_Class__c': 'a014x00000A2WQx',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a064x000004SnB6',
  'ID_Class__c': 'a014x00000A2cwp',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a064x000004SnB6',
  'ID_Class__c': 'a014x00000A2WQx',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a064x000004SnBB',
  'ID_Class__c': 'a014x00000A2cwp',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a064x000004SnBB',
  'ID_Class__c': 'a014x00000A2WQx',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a064x000004SnDM',
  'ID_Class__c': 'a014x00000A2cwp',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a064x000004SnDM',
  'ID_Class__c': 'a014x00000A2WQx',
  'Start_

In [65]:
#inserting 
for rec in class_participant_records:

    record = {
        'ID_Class__c': rec['ID_Class__c'],
        'ID_Student__c': rec['ID_Student__c'],
        'Start_Date__c': rec['Start_Date__c'],
        'End_Date__c': rec['End_Date__c'],
    }
    
    try:
        sf.Class_Participant__c.create(record)
    except Exception as e:
        print(e)

### preparing `Staffassignment` and loading data

In [66]:
sf_staff_records = []
# NameError: name 'sf_Staff_data' is not defined
sf_staff_data = sf.query('SELECT Name, Last_Name__c, Middle_Name__c, First_Name__c, Birth_Date__c, Employee_ID__c FROM Staff__c')
for row in sf_staff_data['records']:
    rec = {
        'ID_Staff__c':row['Name'],
        'Last_Name__c': row['Last_Name__c'],
        'Middle_Name__c': row['Middle_Name__c'],
        'First_Name__c':row['First_Name__c'],
        'Birth_Date__c': row['Birth_Date__c'],
        'Employee_ID__c': row['Employee_ID__c']
    }
    sf_staff_records.append(rec)
sf_staff_records
sf_staff_xwalk = pd.DataFrame(sf_staff_records)
sf_staff_xwalk.head()

,ID_Staff__c,Last_Name__c,Middle_Name__c,First_Name__c,Birth_Date__c,Employee_ID__c
0,a054x0000010hfu,Sanford,Blair,Gemini,1992-04-22,000130109
1,a054x0000010hfp,Luongo,Nico,Darick,1995-07-05,000184220
2,a054x0000010hg4,Rodney,Nicole,Heather,1994-06-07,000159108
3,a054x0000010hfz,Williams,De Angelo,Dartanion,1993-05-21,000160655


In [105]:
#Qeerying the staff tabke from MYSQL
query  ='''
select
    co.CourseCode,
    cl.Section,
    s.EmployeeID,
    sa.role, 
    sa.startdate,
    sa.enddate
from
    staffassignment as sa
    inner join staff s
    on sa.ID_Staff = s.ID_staff
    inner join class cl
    on sa.ID_Class=cl.ID_Class
    inner join course co
    on co.ID_Course=cl.ID_Course
'''
staffassignment_df = pd.read_sql(query, conn)
staffassignment_df.head()

,CourseCode,Section,EmployeeID,role,startdate,enddate
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000184220,Teacher Assistant,2020-09-16,None
1,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000159108,Teacher Assistant,2020-09-16,None
2,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000160655,Instructor,2020-09-16,None


In [106]:
print(staffassignment_df.columns)
print(sf_staff_xwalk.columns)
print(sf_class_xwalk.columns)

Index(['CourseCode', 'Section', 'EmployeeID', 'role', 'startdate', 'enddate'], dtype='object')
Index(['ID_Staff__c', 'Last_Name__c', 'Middle_Name__c', 'First_Name__c',
       'Birth_Date__c', 'Employee_ID__c'],
      dtype='object')
Index(['ID_Class__c', 'ID_Course__c', 'Section__c', 'Course_Code__c'], dtype='object')


In [115]:
sf_class_xwalk

,ID_Class__c,ID_Course__c,Section__c,Course_Code__c
0,a014x00000A2cwz,a004x000003VusQ,GWARL201905WEB3,BC-WEBDEV
1,a014x00000A2cwu,a004x000003VtNK,GWDC201805DATA3,BC-DATAVIZ
2,a014x00000A2cwp,a004x000003VtNK,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
3,a014x00000A2WQx,a004x000003VtNK,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
4,a014x00000A2cx4,a004x000003VusV,GWARL201905UIUX3,BC-UIUX


In [114]:
staff_assignment_load_df = pd.merge(staffassignment_df, sf_class_xwalk, left_on=['CourseCode','Section'], right_on=['Course_Code__c','Section__c'])
staff_assignment_load_df.head()

,CourseCode,Section,EmployeeID,role,startdate,enddate,ID_Class__c,ID_Course__c,Section__c,Course_Code__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000184220,Teacher Assistant,2020-09-16,None,a014x00000A2cwp,a004x000003VtNK,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
1,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000184220,Teacher Assistant,2020-09-16,None,a014x00000A2WQx,a004x000003VtNK,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
2,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000159108,Teacher Assistant,2020-09-16,None,a014x00000A2cwp,a004x000003VtNK,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
3,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000159108,Teacher Assistant,2020-09-16,None,a014x00000A2WQx,a004x000003VtNK,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
4,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000160655,Instructor,2020-09-16,None,a014x00000A2cwp,a004x000003VtNK,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ


In [113]:
staff_assignment_df = pd.merge(staff_assignment_df, sf_class_xwalk, left_on=['CourseCode','Section'], right_on=['Course_Code__c','Section__c'])
staff_assignment_df.head()






,CourseCode,Section,EmployeeID,role,startdate,enddate,ID_Staff__c,Last_Name__c,Middle_Name__c,First_Name__c,Birth_Date__c,Employee_ID__c,ID_Class__c,ID_Course__c,Section__c,Course_Code__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000184220,Teacher Assistant,2020-09-16,None,a054x0000010hfp,Luongo,Nico,Darick,1995-07-05,000184220,a014x00000A2cwp,a004x000003VtNK,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
1,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000184220,Teacher Assistant,2020-09-16,None,a054x0000010hfp,Luongo,Nico,Darick,1995-07-05,000184220,a014x00000A2WQx,a004x000003VtNK,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
2,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000159108,Teacher Assistant,2020-09-16,None,a054x0000010hg4,Rodney,Nicole,Heather,1994-06-07,000159108,a014x00000A2cwp,a004x000003VtNK,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
3,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000159108,Teacher Assistant,2020-09-16,None,a054x0000010hg4,Rodney,Nicole,Heather,1994-06-07,000159108,a014x00000A2WQx,a004x000003VtNK,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
4,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000160655,Instructor,2020-09-16,None,a054x0000010hfz,Williams,De Angelo,Dartanion,1993-05-21,000160655,a014x00000A2cwp,a004x000003VtNK,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ


In [110]:
staff_assignment_df = pd.merge(staffassignment_df, sf_staff_xwalk, left_on='EmployeeID', right_on='Employee_ID__c')
staff_assignment_df.head()

,CourseCode,Section,EmployeeID,role,startdate,enddate,ID_Staff__c,Last_Name__c,Middle_Name__c,First_Name__c,Birth_Date__c,Employee_ID__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000184220,Teacher Assistant,2020-09-16,None,a054x0000010hfp,Luongo,Nico,Darick,1995-07-05,000184220
1,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000159108,Teacher Assistant,2020-09-16,None,a054x0000010hg4,Rodney,Nicole,Heather,1994-06-07,000159108
2,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000160655,Instructor,2020-09-16,None,a054x0000010hfz,Williams,De Angelo,Dartanion,1993-05-21,000160655


In [111]:
staff_assignment_df = staff_assignment_df[['ID_Staff__c','ID_Class__c','role','startdate','enddate']]
staff_assignment_df


KeyError: "['ID_Class__c'] not in index"

In [99]:
staff_assignment_records = sf.query("SELECT Name FROM Staff_Assignment__c")
recs_to_delete = [{'Name': r['Name']} for r in staff_assignment_records['records']]
recs_to_delete
for rec in recs_to_delete:
    try:
        sf.Staff_Assignment__c.delete(rec['Name'])
    except Exception as e:
        print(e)

In [100]:
staff_assignment_df.rename(columns={
    'role':'Role__c',
    'startdate':'StartDate__c',
    'enddate':'EndDate__c'
}, inplace=True)

staff_assignment_df

,ID_Staff__c,ID_Class__c,Role__c,StartDate__c,EndDate__c
0,a054x0000010hfp,a014x00000A2cwp,Teacher Assistant,2020-09-16,None
1,a054x0000010hfp,a014x00000A2WQx,Teacher Assistant,2020-09-16,None
2,a054x0000010hg4,a014x00000A2cwp,Teacher Assistant,2020-09-16,None
3,a054x0000010hg4,a014x00000A2WQx,Teacher Assistant,2020-09-16,None
4,a054x0000010hfz,a014x00000A2cwp,Instructor,2020-09-16,None
5,a054x0000010hfz,a014x00000A2WQx,Instructor,2020-09-16,None


In [101]:
staff_assignment_df['StartDate__c'] = staff_assignment_df['StartDate__c'].astype(str)



In [102]:
staff_assignment_records = staff_assignment_df.to_dict(orient='records')
staff_assignment_records


[{'ID_Staff__c': 'a054x0000010hfp',
  'ID_Class__c': 'a014x00000A2cwp',
  'Role__c': 'Teacher Assistant',
  'StartDate__c': '2020-09-16',
  'EndDate__c': None},
 {'ID_Staff__c': 'a054x0000010hfp',
  'ID_Class__c': 'a014x00000A2WQx',
  'Role__c': 'Teacher Assistant',
  'StartDate__c': '2020-09-16',
  'EndDate__c': None},
 {'ID_Staff__c': 'a054x0000010hg4',
  'ID_Class__c': 'a014x00000A2cwp',
  'Role__c': 'Teacher Assistant',
  'StartDate__c': '2020-09-16',
  'EndDate__c': None},
 {'ID_Staff__c': 'a054x0000010hg4',
  'ID_Class__c': 'a014x00000A2WQx',
  'Role__c': 'Teacher Assistant',
  'StartDate__c': '2020-09-16',
  'EndDate__c': None},
 {'ID_Staff__c': 'a054x0000010hfz',
  'ID_Class__c': 'a014x00000A2cwp',
  'Role__c': 'Instructor',
  'StartDate__c': '2020-09-16',
  'EndDate__c': None},
 {'ID_Staff__c': 'a054x0000010hfz',
  'ID_Class__c': 'a014x00000A2WQx',
  'Role__c': 'Instructor',
  'StartDate__c': '2020-09-16',
  'EndDate__c': None}]

In [103]:
for rec in staff_assignment_records:
    try:
        sf.Staff_Assignment__c.create(rec)
    except Exception as e:
        print(e)

Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Staff_Assignment__c/. Response content: [{'message': 'Required fields are missing: [Class_ID__c]', 'errorCode': 'REQUIRED_FIELD_MISSING', 'fields': ['Class_ID__c']}]
Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Staff_Assignment__c/. Response content: [{'message': 'Required fields are missing: [Class_ID__c]', 'errorCode': 'REQUIRED_FIELD_MISSING', 'fields': ['Class_ID__c']}]
Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Staff_Assignment__c/. Response content: [{'message': 'Required fields are missing: [Class_ID__c]', 'errorCode': 'REQUIRED_FIELD_MISSING', 'fields': ['Class_ID__c']}]
Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Staff_Assignment__c/. Response content: [{'message': 'Required fields are missing: [Class_ID__c]', 'errorCode': 'R

In [491]:
# for rec in staff_assignment_records:
#     record = {}
#     record['ID_Class__c'] = rec['ID_Class__c']
#     record['staff_ID__c'] = rec['staff_ID__c']
#     record['Role__c'] = rec['Role__c']
#     if rec['StartDate__c'] != 0:
#         record['StartDate__c'] = str(rec['StartDate__c'])
#     if rec['EndDate__c'] != None:
#         record['EndDate__c'] = str(rec['EndDate__c'])
    
#     try:
#         sf.Staff_Assignment__c.create(record)
#     except Exception as e:
#         print(e)

Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Staff_Assignment__c/. Response content: [{'message': 'Cannot deserialize instance of double from VALUE_STRING value a054x0000010hND or request may be missing a required field at [line:1, column:34]', 'errorCode': 'JSON_PARSER_ERROR'}]
Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Staff_Assignment__c/. Response content: [{'message': 'Cannot deserialize instance of double from VALUE_STRING value a054x0000010hNS or request may be missing a required field at [line:1, column:34]', 'errorCode': 'JSON_PARSER_ERROR'}]
Malformed request https://d4x000002z413eaa-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Staff_Assignment__c/. Response content: [{'message': 'Cannot deserialize instance of double from VALUE_STRING value a054x0000010hNN or request may be missing a required field at [line:1, column:34]', 'errorCode': 'JSON_PARSER_ERROR'}]
